<a href="https://colab.research.google.com/github/Andrey-Kharlamov/yandex_praktikum/blob/main/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%82%D0%BE%D0%BA%D1%81%D0%B8%D1%87%D0%BD%D1%8B%D1%85%20%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2/toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Токсичные комментарии

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. Для выполнения этой задачи имеется набор данных с разметкой о токсичности правок.

Задача: Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт оснвных библиотек и открытие файла

In [1]:
import pandas as pd
import numpy as np
import re 

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem.lancaster import LancasterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC


import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
url = 'https://drive.google.com/uc?id=' + 'https://drive.google.com/file/d/18DxwFo9hNZW9ezk7FAfJ08kKIAn0HuY0/view?usp=sharing'.split('/')[-2]
data = pd.read_csv(url)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data['text'].duplicated().sum()

0

In [5]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Создам функцию для подготовки текста

In [6]:
def preprocess_text(texts):
    stop_words = set(stopwords.words('english'))
    regex = re.compile('[^a-z A-Z]')
    preprocess_texts = []
    
    for i in  tqdm.tqdm(range(len(texts))):
        text = texts[i].lower()
        text = regex.sub(' ', text) #Удаление символов, которые не являются английскими в верехнем/нижнем регистрах
        word_tokens = word_tokenize(text) 
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        preprocess_texts.append( ' '.join(filtered_sentence))
    
    return preprocess_texts

In [7]:
train, test = train_test_split(data, test_size=0.2, stratify=data['toxic'], random_state=12345)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)


In [8]:
train['text'] = preprocess_text(train['text'])

100%|██████████| 127656/127656 [01:05<00:00, 1963.15it/s]


In [9]:
test['text'] = preprocess_text(test['text'])

100%|██████████| 31915/31915 [00:11<00:00, 2685.01it/s]


Функция для стемминга текта

In [10]:
def stemming_texts(texts):
    st = LancasterStemmer()
    stem_text = []
    for text in tqdm.tqdm(texts):
        word_tokens = word_tokenize(text)
        stem_text.append(' '.join([st.stem(word) for word in word_tokens]))
    return stem_text

In [11]:
train['text'] = stemming_texts(train.text)

100%|██████████| 127656/127656 [01:52<00:00, 1137.46it/s]


In [12]:
test['text'] = stemming_texts(test.text)

100%|██████████| 31915/31915 [00:28<00:00, 1130.29it/s]


In [13]:
count_vect = CountVectorizer()

In [14]:
features_train_bow = count_vect.fit_transform(train.text)
features_test_bow = count_vect.transform(test.text)

Векторизация текста для *TF -IDF*

In [15]:
vectorizer_tf_idf = TfidfVectorizer()

In [16]:
features_train_tfidf = vectorizer_tf_idf.fit_transform(train.text)
features_test_tfidf = vectorizer_tf_idf.transform(test.text)

Векторизация текста мешок слов с n-граммой

In [17]:
count_vect_bow_ngram = CountVectorizer(ngram_range=(1, 2))

In [18]:
features_train_bow_ngram = count_vect_bow_ngram.fit_transform(train.text)
features_test_bow_ngram = count_vect_bow_ngram.transform(test.text)

Векторизация текста *TF-IDF* с N-граммой

In [19]:
vectorizer_tf_idf_ngramm = TfidfVectorizer(ngram_range=(1, 2))

In [20]:
features_train_tfidf_ngramm = vectorizer_tf_idf_ngramm.fit_transform(train.text)
features_test_tf_idf_ngramm = vectorizer_tf_idf_ngramm.transform(test.text)

Целевая переменная

In [21]:
target_train = train['toxic']
target_test = test['toxic']

### Вывод
В ходе подготовки текста были выполнены следующие действия:
* загружены данные и получена общая информация
* из теста удалены лишние символы и стоп-слова (токенизирован)
* слова текста приведены к начальной форме
* тексты преобразованы в вектора.

## Обучение

**Подбор параметров для логистической регрессии для модели *мешок слов*.**

In [22]:
model_reg_bow = LogisticRegression(random_state=12345, solver='saga')

In [23]:
grid_reg_params = {'C' : [0.1, 1, 10, 50],
           'max_iter':[1000, 2000]}

In [24]:
grid_reg_bow = GridSearchCV(model_reg_bow, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [25]:
grid_reg_bow.fit(features_train_bow, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=12345, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [26]:
grid_reg_bow.best_params_

{'C': 50, 'max_iter': 2000}

In [27]:
grid_reg_bow.best_score_

0.6345126494738186

**Подбор параметров для модели svc с мешком слов**

In [28]:
model_svc_bow = LinearSVC(random_state=12345, penalty='l1', dual=False)

In [29]:
grid_svc_bow = GridSearchCV(model_svc_bow, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [30]:
grid_svc_bow.fit(features_train_bow, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


GridSearchCV(cv=3,
             estimator=LinearSVC(dual=False, penalty='l1', random_state=12345),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [ ]:
grid_svc_bow.best_params_, grid_svc_bow.best_score_

**Ngramm для логистической регрессии *bow***

In [ ]:
model_reg_bow_ngramm = LogisticRegression(random_state=12345, solver='saga')

In [ ]:
grid_reg_bow_ngramm = GridSearchCV(model_reg_bow_ngramm, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [34]:
grid_reg_bow_ngramm.fit(features_train_bow_ngram, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=12345, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [35]:
grid_reg_bow_ngramm.best_params_, grid_reg_bow_ngramm.best_score_

({'C': 1, 'max_iter': 2000}, 0.5867548667934253)

**NGramm для модели svc *bow***

In [36]:
model_svc_bow_ngramm = LinearSVC(random_state=12345, penalty='l1', dual=False)

In [37]:
grid_svc_bow_ngramm = GridSearchCV(model_svc_bow_ngramm, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [38]:
grid_svc_bow_ngramm.fit(features_train_bow_ngram, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


GridSearchCV(cv=3,
             estimator=LinearSVC(dual=False, penalty='l1', random_state=12345),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [39]:
grid_svc_bow_ngramm.best_params_, grid_svc_bow_ngramm.best_score_

({'C': 1, 'max_iter': 1000}, 0.7581499394597421)

**Модель логистической регрессии TF-IDF**

In [40]:
model_reg_tfidf = LogisticRegression(random_state=12345, solver='saga')

In [41]:
grid_reg_tfidf = GridSearchCV(model_reg_tfidf, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [42]:
grid_reg_tfidf.fit(features_train_tfidf, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=12345, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [43]:
grid_reg_tfidf.best_params_, grid_reg_tfidf.best_score_

({'C': 10, 'max_iter': 1000}, 0.7665827133676698)

**Модель svc TF-IDF**

In [44]:
model_svc_tfidf = LinearSVC(random_state=12345, penalty='l1', dual=False)

In [45]:
grid_svc_tfidf = GridSearchCV(model_svc_tfidf, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [46]:
grid_svc_tfidf.fit(features_train_tfidf, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3,
             estimator=LinearSVC(dual=False, penalty='l1', random_state=12345),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [47]:
grid_svc_tfidf.best_params_, grid_svc_tfidf.best_score_

({'C': 1, 'max_iter': 1000}, 0.7763229358136577)

**Модель логистической регрессии TF -IDF N-gramm**

In [48]:
model_reg_tfidf_ngramm = LogisticRegression(random_state=12345, solver='saga')

In [49]:
grid_reg_tfidf_ngramm = GridSearchCV(model_reg_tfidf_ngramm, param_grid=grid_reg_params, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [50]:
grid_reg_tfidf_ngramm.fit(features_train_tfidf_ngramm, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=12345, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [51]:
grid_reg_tfidf_ngramm.best_params_, grid_reg_tfidf_ngramm.best_score_

({'C': 50, 'max_iter': 1000}, 0.7387836633812195)

**Модель svc TF-IDF N-gramm**

In [52]:
model_svc_tfidf_ngramm = LinearSVC(random_state=12345, penalty='l1', dual=False)

In [53]:
grid_svc_tfidf_ngramm = GridSearchCV(model_svc_tfidf_ngramm, param_grid=grid_reg_params, scoring='f1', cv=5, verbose=1, n_jobs=-1)

In [54]:
grid_svc_tfidf_ngramm.fit(features_train_tfidf_ngramm, target_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=LinearSVC(dual=False, penalty='l1', random_state=12345),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 50], 'max_iter': [1000, 2000]},
             scoring='f1', verbose=1)

In [55]:
grid_svc_tfidf_ngramm.best_params_, grid_svc_tfidf_ngramm.best_score_

({'C': 1, 'max_iter': 1000}, 0.7751588460704499)

Лучшие результаты оказались у модели, созданной с помощью классификации опорных векторов для TF-IDF. На нем и посмотрю результат на тестовой выборке для модели с n-граммой и без.

In [56]:
model_svc_tfidf = LinearSVC(random_state=12345, penalty='l1', dual=False, C=1, max_iter=1000)

In [57]:
%%time
model_svc_tfidf.fit(features_train_tfidf, target_train)

CPU times: user 3.25 s, sys: 8.93 ms, total: 3.26 s
Wall time: 3.27 s


LinearSVC(C=1, dual=False, penalty='l1', random_state=12345)

In [58]:
predictions = pd.Series(model_svc_tfidf.predict(features_test_tfidf))
score = f1_score(target_test, predictions)
print('F1 мера для SVC модели с использованием TF-IDF: {:.3f}'.format(score))

F1 мера для SVC модели с использованием TF-IDF: 0.785


In [59]:
model_svc_tfidf_ngramm = LinearSVC(random_state=12345, penalty='l1', dual=False)

In [60]:
model_svc_tfidf_ngramm.fit(features_train_tfidf_ngramm, target_train)

LinearSVC(dual=False, penalty='l1', random_state=12345)

In [61]:
predictions_svc = model_svc_tfidf_ngramm.predict(features_test_tf_idf_ngramm)

In [62]:
print('F1 мера для SVC модели с использованием TF-IDF и n-граммой: {:.3f}'.format(f1_score(target_test, predictions_svc)))

F1 мера для SVC модели с использованием TF-IDF и n-граммой: 0.795


### Вывод
В данном разделе были обучены модели логистической регрессии и SVC. Вторая показала более лучшие результаты. Также можно выделить, что используя TF-IDF для обеих моделей были получены более высокие результаты. Использование биграммы еще более улучшило F1-меру.

## Выводы

В проекте необходимо было разработать инструмент для интернет магазина *Викишоп*, который бы искал токсичные комментарии. Для этого необходимо было обучить модель с F1-мерой не ниже 0.75. Такая метрика использовалась из-за дисбаланса классоа в целевой переменной. Токсичных комментариев гораздо меньше, чем нормальных. Для обучения модели данные были токенезированы, убраны стоп слова и проведена лемматизация. После чего текст был преобразован в векторный формат, по которому и обучались модели. Данные преобразовывались несколькими способами:
* модель *мешок слов*
* модель *мешок слов* с биграммой
* модель TF-IDF
* модель TF-IDF с биграммой

Для обучения были выбраны модели логистической регрессии и опорных векторов. SVC показала более высокие результаты. Наилучший результат показала модель SVC TF-IDF с биграммой. У нее значение F1-меры на тестовой выборке 0.795.